In [ ]:
!git clone https://github.com/ultralytics/yolov5 

In [2]:
# Load YOLOv5 model
import torch
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5x')
#yolo_model = torch.hub.load('ultralytics/yolov5', 'yolo5s')

Using cache found in C:\Users\vivek/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-2 Python-3.12.1 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

100%|██████████| 40.8M/40.8M [00:03<00:00, 13.3MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [4]:
from PIL import Image
import os

# Set the directory containing your images -C:\Users\vivek\Desktop\FRND\data\fall_dataset\images\train
directory = 'C:/Users/vivek/Desktop/FRND/data/fall_dataset/images/train'
#C:\Users\vivek\Desktop\FRND\data\pp data\train\images
output_directory = 'C:/Users/vivek/Desktop/FRND/data/pp data/train/images'

# Set the desired size
new_size = (640, 480)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        img_path = os.path.join(directory, filename)
        img = Image.open(img_path)
        # Resize the image using LANCZOS resampling (formerly ANTIALIAS)
        img = img.resize(new_size, Image.Resampling.LANCZOS)
        # Save the resized image to the output directory
        img.save(os.path.join(output_directory, filename))

print("All images have been resized and saved to", output_directory)


All images have been resized and saved to C:/Users/vivek/Desktop/FRND/data/pp data/train/images


In [5]:
from PIL import Image
import os

# Set the directory containing your images -C:\Users\vivek\Desktop\FRND\data\fall_dataset\images\train
directory = 'C:/Users/vivek/Desktop/FRND/data/fall_dataset/images/val'
#C:\Users\vivek\Desktop\FRND\data\pp data\train\images
output_directory = 'C:/Users/vivek/Desktop/FRND/data/pp data/val/images'

# Set the desired size
new_size = (640, 480)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        img_path = os.path.join(directory, filename)
        img = Image.open(img_path)
        # Resize the image using LANCZOS resampling (formerly ANTIALIAS)
        img = img.resize(new_size, Image.Resampling.LANCZOS)
        # Save the resized image to the output directory
        img.save(os.path.join(output_directory, filename))

print("All images have been resized and saved to", output_directory)


All images have been resized and saved to C:/Users/vivek/Desktop/FRND/data/pp data/val/images


In [8]:
import torch
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import os

# Load a more accurate YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

def process_images(input_folder, output_folder, labels_dir, output_csv):
    annotations = []
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            frame = cv2.imread(image_path)
            h, w, _ = frame.shape  # Get image dimensions

            # Apply CLAHE
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            enhanced_gray = clahe.apply(gray)
            frame = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            # Inference
            results = model(img, size=640)
            detections = results.xyxy[0]
            person_detections = detections[detections[:, 5] == 0]

            # Process detections
            if len(person_detections) > 0:
                for *xyxy, conf, cls in person_detections:
                    x1, y1, x2, y2 = map(int, xyxy)
                    x_center = ((x1 + x2) / 2) / w
                    y_center = ((y1 + y2) / 2) / h
                    box_width = (x2 - x1) / w
                    box_height = (y2 - y1) / h
                    class_id = 0 if filename[0].lower() == 'f' else 1
                    annotation = {
                        "Filename": filename,
                        "class_id": class_id,
                        "x_center": x_center,
                        "y_center": y_center,
                        "width": box_width,
                        "height": box_height,
                        "Confidence": float(conf)
                    }
                    annotations.append(annotation)

                    # Draw boxes and labels on the image
                    label = 'Fall' if class_id == 0 else 'No Fall'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                    # Write YOLO format annotation to .txt file
                    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"
                    label_filename = f"{os.path.splitext(filename)[0]}.txt"
                    label_file_path = os.path.join(labels_dir, label_filename)
                    with open(label_file_path, 'a') as file:
                        file.write(yolo_format)

                # Save the annotated image only if persons are detected
                cv2.imwrite(os.path.join(output_folder, 'annotated_' + filename), frame)

    # Save annotations to CSV
    df = pd.DataFrame(annotations)
    df.to_csv(output_csv, index=False)

input_folder = 'C:/Users/vivek/Desktop/FRND/data/pp data/train/images'
output_folder = 'C:/Users/vivek/Desktop/FRND/data/pp data/new annotated'
labels_dir = 'C:/Users/vivek/Desktop/FRND/data/pp data/train/labels'
output_csv = 'C:/Users/vivek/Desktop/FRND/data/pp data/train.csv'
process_images(input_folder, output_folder, labels_dir, output_csv)


Using cache found in C:\Users\vivek/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-2 Python-3.12.1 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


In [9]:
import torch
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import os

# Load a more accurate YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

def process_images(input_folder, output_folder, labels_dir, output_csv):
    annotations = []
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_folder, filename)
            frame = cv2.imread(image_path)
            h, w, _ = frame.shape  # Get image dimensions

            # Apply CLAHE
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            enhanced_gray = clahe.apply(gray)
            frame = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            # Inference
            results = model(img, size=640)
            detections = results.xyxy[0]
            person_detections = detections[detections[:, 5] == 0]

            # Process detections
            if len(person_detections) > 0:
                for *xyxy, conf, cls in person_detections:
                    x1, y1, x2, y2 = map(int, xyxy)
                    x_center = ((x1 + x2) / 2) / w
                    y_center = ((y1 + y2) / 2) / h
                    box_width = (x2 - x1) / w
                    box_height = (y2 - y1) / h
                    class_id = 0 if filename[0].lower() == 'f' else 1
                    annotation = {
                        "Filename": filename,
                        "class_id": class_id,
                        "x_center": x_center,
                        "y_center": y_center,
                        "width": box_width,
                        "height": box_height,
                        "Confidence": float(conf)
                    }
                    annotations.append(annotation)

                    # Draw boxes and labels on the image
                    label = 'Fall' if class_id == 0 else 'No Fall'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                    # Write YOLO format annotation to .txt file
                    yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"
                    label_filename = f"{os.path.splitext(filename)[0]}.txt"
                    label_file_path = os.path.join(labels_dir, label_filename)
                    with open(label_file_path, 'a') as file:
                        file.write(yolo_format)

                # Save the annotated image only if persons are detected
                cv2.imwrite(os.path.join(output_folder, 'annotated_' + filename), frame)

    # Save annotations to CSV
    df = pd.DataFrame(annotations)
    df.to_csv(output_csv, index=False)

input_folder = 'C:/Users/vivek/Desktop/FRND/data/pp data/val/images'
output_folder = 'C:/Users/vivek/Desktop/FRND/data/pp data/new vannotated'
labels_dir = 'C:/Users/vivek/Desktop/FRND/data/pp data/val/labels'
output_csv = 'C:/Users/vivek/Desktop/FRND/data/pp data/val.csv'
process_images(input_folder, output_folder, labels_dir, output_csv)


Using cache found in C:\Users\vivek/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-2 Python-3.12.1 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit 0.550s exceeded
WARNING  NMS time limit

In [1]:
!python yolov5/train.py --img 640 --batch 14 --epochs 40 --data data.yaml --weights yolov5s.pt --cfg yolov5/models/yolov5s.yaml --project yolov5 --name finalfall --resume

train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=data.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=40, batch_size=14, imgsz=640, rect=False, resume=True, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5\data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5, name=finalfall, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-305-g4456c953 Python-3.12.1 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.

In [21]:
!python yolov5/detect.py --weights yolov5/finalfall/weights/best.pt --img 640 --conf 0.7 --source test/ --iou 0.9

detect: weights=['yolov5/finalfall/weights/best.pt'], source=test/, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.7, iou_thres=0.9, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-305-g4456c953 Python-3.12.1 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/19 C:\Users\vivek\Desktop\FRND\fall\test\1.jpg: 544x640 2 No Falls, 89.0ms
image 2/19 C:\Users\vivek\Desktop\FRND\fall\test\2.jpg: 576x640 (no detections), 90.3ms
image 3/19 C:\Users\vivek\Desktop\FRND\fall\test\3.png: 480x640 1 Fall, 88.8ms
image 4/19 C:\Users\vivek\Desktop\FRND\fall\test\4.jpg: 